Import libraries

In [13]:
import pandas as pd
from pulp import *
import random

Function for calculating the TTE

In [14]:
def calculate_tte():
    # Take age, gender, height and weight from the user
    age = 28
    gender = 'f'
    height = 160
    weight = 55

    # Calculate BMR
    if gender.lower() == 'm':
        bmr = 88.362 + (13.397 * weight) + (4.799 * height) - (5.677 * age)
    else:
        bmr = 447.593 + (9.247 * weight) + (3.098 * height) - (4.330 * age)

    # Calculate TTE (because they want to be active, I chose 1.7)
    # TTE is stands on Total Energy Expenditure
    calculated_tte = bmr * 1.7

    return calculated_tte
tte = calculate_tte()

Read the USDA dataset

In [15]:
df = pd.read_csv('USDA.csv')
df = df.dropna(axis=0, how='any')
df.drop_duplicates(subset=None, inplace=True)

Obtain map between ingredients and nutrition

In [16]:
ingredients = list(df['Description'])
ingredient_calorie_map = dict(zip(ingredients, df['Calories']))
ingredient_protein_map = dict(zip(ingredients, df['Protein']))
ingredient_total_fat_map = dict(zip(ingredients, df['TotalFat']))
ingredient_carbohydrate_map = dict(zip(ingredients, df['Carbohydrate']))
ingredient_iron_map = dict(zip(ingredients, df['Iron']))

Define minimum constraints for Fat, Carbohydrate, Protein, and Iron for athletes and someone who wants to lose weight

In [17]:
min_calories = tte - 200
min_fat = 0.25 * tte / 9
min_carbohydrate = 0.50 * tte / 4
min_protein = 0.15 * tte / 4
min_iron = 0.0043 * tte

max_calories = tte + 200
max_fat = 0.4 * tte / 9
max_carbohydrate = 0.7 * tte / 4
max_protein = 0.4 * tte / 4
max_iron = 0.009 * tte

Define linear programming problem and also its constrains

In [18]:
problem = LpProblem('Diet_plan', LpMinimize)
food_vars = LpVariable.dicts('Food', ingredients, 0, cat='Integer')

# Calories
problem += lpSum([ingredient_calorie_map[f] * food_vars[f] for f in ingredients]) >= min_calories, 'CaloriesMinimum'
problem += lpSum([ingredient_calorie_map[f] * food_vars[f] for f in ingredients]) <= max_calories, 'CaloriesMaximum'

# Fat
problem += lpSum([ingredient_total_fat_map[f] * food_vars[f] for f in ingredients]) >= min_fat, 'FatMinimum'
problem += lpSum([ingredient_total_fat_map[f] * food_vars[f] for f in ingredients]) <= max_fat, 'FatMaximum'

# Carbohydrate
problem += lpSum(
    [ingredient_carbohydrate_map[f] * food_vars[f] for f in ingredients]) >= min_carbohydrate, 'CarbsMinimum'
problem += lpSum(
    [ingredient_carbohydrate_map[f] * food_vars[f] for f in ingredients]) <= max_carbohydrate, 'CarbsMaximum'

# Protein
problem += lpSum([ingredient_protein_map[f] * food_vars[f] for f in ingredients]) >= min_protein, 'ProteinMinimum'
problem += lpSum([ingredient_protein_map[f] * food_vars[f] for f in ingredients]) <= max_protein, 'ProteinMaximum'

# Iron
problem += lpSum([ingredient_iron_map[f] * food_vars[f] for f in ingredients]) >= min_iron, 'IronMinimum'
problem += lpSum([ingredient_iron_map[f] * food_vars[f] for f in ingredients]) <= max_iron, 'IronMaximum'

problem.solve(PULP_CBC_CMD(msg=False))

1

Define list of meals and also initialize a dict for map meals to their ingredients

In [19]:
meals = ['Breakfast', 'Morning snack', 'Lunch', 'Afternoon snack', 'Dinner']
meals_dict = {
    'Breakfast': [],
    'Morning snack': [],
    'Lunch': [],
    'Afternoon snack': [],
    'Dinner': []
}

If the solver could reach to optimal solve for the problem we define above, we can arrange the meals

In [20]:
if LpStatus[problem.status] == 'Optimal':
    total_calories = 0
    total_fat = 0
    total_carbohydrate = 0
    total_protein = 0
    total_iron = 0

    # Obtain ingredients with varValue more than zero
    suitable_ingredients_list = []
    for ingredient in ingredients:
        food_var = food_vars[ingredient]
        if food_var.varValue is not None and food_var.varValue > 0:
            suitable_ingredients_list.append(ingredient)

    random.shuffle(suitable_ingredients_list)

    # Put the ingredients in meals dict almost evenly
    for index, ingredient in enumerate(suitable_ingredients_list):
        food_var = food_vars[ingredient]
        ingredient_amount = food_var.varValue
        ing_fat = ingredient_total_fat_map.get(ingredient, 0) * ingredient_amount
        ing_calories = ingredient_calorie_map.get(ingredient, 0) * ingredient_amount
        ing_carbohydrate = ingredient_carbohydrate_map.get(ingredient, 0) * ingredient_amount
        ing_protein = ingredient_protein_map.get(ingredient, 0) * ingredient_amount
        ing_iron = ingredient_iron_map.get(ingredient, 0) * ingredient_amount

        total_fat += ing_fat
        total_calories += ing_calories
        total_carbohydrate += ing_carbohydrate
        total_protein += ing_protein
        total_iron += ing_iron
        meals_dict[meals[index % len(meals)]].append(
            [ingredient, ingredient_amount, ing_fat, ing_calories, ing_carbohydrate, ing_protein, ing_iron])

    # Find ingredient which doesn't break the constraints
    additional_items = []
    for ingredient in ingredients:
        food_var = food_vars[ingredient]
        ingredient_amount = 1.0
        ing_fat = ingredient_total_fat_map.get(ingredient, 0) * ingredient_amount
        ing_calorie = ingredient_calorie_map.get(ingredient, 0) * ingredient_amount
        ing_carbohydrate = ingredient_carbohydrate_map.get(ingredient, 0) * ingredient_amount
        ing_protein = ingredient_protein_map.get(ingredient, 0) * ingredient_amount
        ing_iron = ingredient_iron_map.get(ingredient, 0) * ingredient_amount
        if ing_fat + total_fat < max_fat and ing_carbohydrate + total_carbohydrate < max_carbohydrate \
                and ing_protein + total_protein < max_protein and ing_iron + total_iron < max_iron \
                and ing_calorie + total_calories < max_calories:
            total_fat += ing_fat
            total_calories += ing_calorie
            total_carbohydrate += ing_carbohydrate
            total_protein += ing_protein
            total_iron += ing_iron
            additional_items.append(
                [ingredient, ingredient_amount, ing_fat, ing_calorie, ing_carbohydrate, ing_protein, ing_iron])
    # Add additional items to the meals almost evenly
    for index, item in enumerate(additional_items):
        meals_dict[meals[index % len(meals)]].append(item)

    # Print the output and calculate every meal nutritional content
    for meal, data in meals_dict.items():
        meal_fat = 0
        meal_calories = 0
        meal_carbohydrate = 0
        meal_protein = 0
        meal_iron = 0
        print(f'- {meal}')
        for item in data:
            print(f'    - {item[1]} {item[0]}')
            meal_fat += item[2]
            meal_calories += item[3]
            meal_carbohydrate += item[4]
            meal_protein += item[5]
            meal_iron += item[6]
        print(f'(Nutritional Content: Calories: {meal_calories}, Fat: {meal_fat}, Carbohydrate: {meal_carbohydrate}, Protein: {meal_protein}, Iron: {meal_iron})')
        print()

    print(
        f'Total Nutritional Content (for all of meals together): Calories: {total_calories}, Fat: {total_fat}, Carbohydrate: {total_carbohydrate}, Protein: {total_protein}, Iron: {total_iron}')
else:
    print('There is no solution with your input')


- Breakfast
    - 1.0 SYRUPS,DIETETIC
    - 4.0 FAT FREE ICE CRM,NO SUGAR ADDED,FLAVORS OTHER THAN CHOC
    - 1.0 CHEESE,COTTAGE,CRMD,LRG OR SML CURD
    - 1.0 CARBONATED BEV,CLUB SODA
    - 1.0 WATER,TAP,DRINKING
(Nutritional Content: Calories: 654.0, Fat: 4.3, Carbohydrate: 164.34, Protein: 29.560000000000002, Iron: 0.08)

- Morning snack
    - 555.0 CARBONATED BEV,CLUB SODA
    - 1.0 CHEESE,COTTAGE,CRMD,W/FRUIT
    - 1.0 CARBONATED BEV,LO CAL,OTHER THAN COLA OR PEPPER,WO/ CAFFEINE
    - 1.0 WATER,TAP,MUNICIPAL
(Nutritional Content: Calories: 97.0, Fat: 3.85, Carbohydrate: 4.61, Protein: 10.79, Iron: 5.75)

- Lunch
    - 34.0 SOUP,PEPPERPOT,CND,PREP W/ EQ VOLUME H2O
    - 1.0 MILK,FILLED,FLUID,W/LAURIC ACID OIL
    - 1.0 CARB BEV,LO CAL,OTHR THN COLA OR PEPPER,W/ ASPRT,CONTNS CAFF
    - 1.0 WATER,BTLD,GENERIC
(Nutritional Content: Calories: 1457.0, Fat: 66.30000000000001, Carbohydrate: 132.24, Protein: 89.78999999999999, Iron: 12.33)

- Afternoon snack
    - 1.0 ALCOHOLIC BEV,DISTILL